![logo](img/logo.png)

# Dealing with Named common

The Named common stays active for the life of he U2 process.  

There is no direct access to the BASIC Named Common variables, this notebook shows a way to get and set the variables from Python.

## Connect to U2 session

In [2]:
import os
xdemoPath = "c:\\U2\\ud83\\XDEMO"
os.chdir(xdemoPath)
print(os.getcwd())
import u2py
import u2support

c:\U2\ud83\XDEMO



## Running a Program that has a Named common

Lets start by looking at he code.



In [6]:
rec = u2support.U2READ("PBP", "SET_BASE", "")
for eachline, ref in rec:
    print(eachline)



*
COMMON /MIKE/ FILEVAR, DATA_ARRAY(10), MY_DYN_ARRAY
* NOTE THE NAME COMMON ABOVE DEFINES 12 VARIABLES
* If FILEVAR is a file variable, we can not send it to python
* the dim array is the same as having 10 variables, and those can not be file variables.
**
OPEN "MEMBERS" TO FILEVAR ELSE STOP "NG"
*
MAT DATA_ARRAY = "dim data"
DATA_ARRAY(7) = "MIKE"
DATA_ARRAY(5) = "1":@AM:"2":@AM:"3"
*
MY_DYN_ARRAY = "dynamic":@VM:"array"


### Running the program to set the named common

In [14]:
cmd = u2py.Command("RUN PBP SET_BASE")
cmd.run()

### We then will need a subroutine to get the named common

For Example:

In [15]:
rec = u2support.U2READ("PBP", "GET_NC_MIKE", "")
for eachline, ref in rec:
    print(eachline)

SUBROUTINE GET_NC_MIKE(  GET_THIS, THE_ARRAY )
COMMON /MIKE/ FILE_VAR, DIMARRAY(10), DYNARRAY
BEGIN CASE
  CASE GET_THIS = "DIMARRAY"
    MATBUILD THE_ARRAY FROM DIMARRAY USING "~"
    THE_ARRAY = LOWER(THE_ARRAY)
    CONVERT "~" TO @AM IN THE_ARRAY
  CASE GET_THIS = "DYNARRAY"
    THE_ARRAY = DYNARRAY
END CASE
RETURN


## For my example we can call the subroutine and request either:
##### "DIMARRAY" - Which gets all the elements in the dimentioned array.
#### or
##### "DYNARRAY" - Gets the DYNARRAY dynamic array from the named common.

In [23]:
sub = u2py.Subroutine("GET_NC_MIKE",2 )
sub.args[0]="DIMARRAY"
sub.call()
print("The " + str(sub.args[0]) + " = " + str(sub.args[1]))

sub.args[0]="DYNARRAY"
sub.call()
print("The " + str(sub.args[0]) + " = " + str(sub.args[1]))


The DIMARRAY = dim data�dim data�dim data�dim data�1�2�3�dim data�MIKE�dim data�dim data�dim data
The DYNARRAY = dynamic�array


#### Instead of getting the entire Dimentioned array, it is possible to get a specific element.

Yet, you need to make sure you are not getting a file pointer, since these can not be used in Python.

In [24]:
rec = u2support.U2READ("PBP", "GET_ONE_ELEMENT", "")
for eachline, ref in rec:
    print(eachline)

$BASICTYPE "P"
SUBROUTINE GET_ONE_ELEMENT( POS, THE_DATA )
COMMON /MIKE/ MY_ARRAY(12)
*
THE_DATA = MY_ARRAY(POS)
IF FILEINFO(THE_DATA, 0) THEN
	THE_DATA = "mv_file_pointer"
END
RETURN 


##### Note that for UniData, we need the program to be compiled for BASICTYPE 'P', so we can reference the named common two different ways.

i.e.
##### COMMON /MIKE/ MY_ARRAY(12)
or
##### COMMON /MIKE/ FILE_VAR, DIMARRAY(10), DYNARRAY

This way we can ask for a specific named COMMON variable based on it position. 

In [30]:
sub = u2py.Subroutine("GET_ONE_ELEMENT",2 )
for each in range(1, 13):
    sub.args[0]=each
    sub.call()
    print("variable " + str(sub.args[0]) + " = " + str(sub.args[1]))


variable 1 = mv_file_pointer
variable 2 = dim data
variable 3 = dim data
variable 4 = dim data
variable 5 = dim data
variable 6 = 1�2�3
variable 7 = dim data
variable 8 = MIKE
variable 9 = dim data
variable 10 = dim data
variable 11 = dim data
variable 12 = dynamic�array


Final thought, this is just the start of how you can manage the access to the named common.  

With similar subroutines, it would be possible to create an Python object that would get the needed common varaibles, and set them, if/when the object changes in python.

But that is an example for another time.